In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
%cd /path_to_code/SHAnTIE/
import astropy.units as u

In [ ]:
import numpy as np
import healpy as hp
import h5py
import copy
import sys
from intensity_mapping_refactor import SkyMap3D

In [ ]:
#Load binned CMASS galaxies.
BOSS_path = 'path_to_binned_BOSS_galaxy_files'

with h5py.File(BOSS_path + 'CMASS_random1_galaxies_dzp1.h5', 'r') as f:
    print(f.keys())
    CMASS_randoms = np.array(f['hitmap'])
with h5py.File(BOSS_path + 'CMASS_real_galaxies_with_wtot_weighting.h5', 'r') as f:
    print(f.keys())
    redshift_edges = np.array(f['z_edges'])
    CMASS_reals = np.array(f['hitmap'])

#Create SkyMap3D object to hold CMASS maps and weights.
#The .from_gal_counts constructor makes overdensity maps from the galaxy number counts and the random galaxies.
#Using sel_func_normalized = False enforces the number of random galaxies to be equal to the number of reals.
#This option is needed because there are 50 simulated galaxy realizations in the randoms.
CMASS_SSM3D = SkyMap3D.SphericalSkyMap3D.from_gal_counts(CMASS_reals, CMASS_randoms, z_edges = redshift_edges,
                                                         sel_func_normalized=False, win_func = None)

#Plot the weights at redshift bin 2.
hp.mollview(CMASS_SSM3D.weights[2,:],
        title = 'Selection function at ' + str(CMASS_SSM3D.z_edges[2]) + '<z<' + str(CMASS_SSM3D.z_edges[3]))
#Plot the overdensities at redshift bin 2.
hp.mollview(CMASS_SSM3D.map_array[2,:],
        title = 'Overdensity at ' + str(CMASS_SSM3D.z_edges[2]) + '<z<' + str(CMASS_SSM3D.z_edges[3]))

In [ ]:
#Create TomographicPair object to hold 2 copies of SkyMap3D object and compute angular power spectrum.
CMASSxCMASS = SkyMap3D.TomographicPair(CMASS_SSM3D, CMASS_SSM3D)
#Choose angular bin size for Cl computation.
bin_size = 30
#Compute power spectrum cl for all pairs of redshift bins.
CMASSxCMASS.compute_cl_zz(bin_size = bin_size, save_unbinned_pcl = True)

In [ ]:
#Quick tour of main result arrays.

#CMASSxCMASS.cl_zz is the binned angular power spectrum, where unmixing has been attempted via the binned mixing matrix.
#Computation done for all pairs of redshifts.
#The shape of cl_zz is (# angular bins, # redshift bins, # redshift bins).
print(CMASSxCMASS.cl_zz.shape)

#CMASSxCMASS.pcl_zz_binned is the binned pseudoCl spectrum, (angular power spectrum of overdensities times weights).
#No unmixing is performed. Computation done for all pairs of redshifts.
#The shape of pcl_zz_binned is (# angular bins, # redshift bins, # redshift bins).
print(CMASSxCMASS.pcl_zz_binned.shape)

#CMASSxCMASS.pcl_zz_unbinned is the unbinned pseudoCl spectrum, (angular power spectrum of overdensities times weights).
#No unmixing is performed. Computation done for all pairs of redshifts.
#The shape of pcl_zz_unbinned is (# of ells (3*Nside), # redshift bins, # redshift bins).
print(CMASSxCMASS.pcl_zz_unbinned.shape)

In [ ]:
#Save file.

#The save function will save all numpy array attributes of the object, like map_array, weights, cl_zz, etc.
CMASSxCMASS.write_to_hdf5('temp.h5')

In [ ]:
#Load file.

#The load function will load all the saved numpy array attributes like map_array, weights, cl_zz, etc.
Loaded_CMASSxCMASS = SkyMap3D.TomographicPair('temp.h5')

In [ ]:
#Make some plots.

from intensity_mapping_refactor import plotting_tools as pt

#Plot all the overdensity maps.
pt.plot_maps(CMASSxCMASS.maps1.map_array, fname='temp.pdf')

#Plot cl_zz cubes. Each cube is at a given ell, and shows Cl at all pairs of redshifts.
pt.plot_cl_zz_cubes(CMASSxCMASS, fname='temp.pdf')

In [ ]:
#Plot cl as a function of ell for all redshift bin pairs.

#Define function for computing approximate error diagonals from C_ell_zz.
#These error bars are only approximate.

def f_sky(weights1, weights2):
    factor = np.sum(weights1*weights2)**2.
    factor /= np.sum(weights1**2*weights2**2.)*float(np.size(weights1))
    return factor

def compute_error_diagonals(this_cl_object, auto1_cl_zz = None, auto2_cl_zz = None, cross = False):
    fsky = f_sky(this_cl_object.maps1.weights[0,:], this_cl_object.maps2.weights[0,:])
    pre_factor = (this_cl_object.ell_bin_size*fsky*(2*this_cl_object.ells+1))**(-1)
    if not cross:
        this_cl_zz_auto_diag = np.einsum('ijj->ij', this_cl_object.cl_zz)
        var = pre_factor[:,None,None]*(np.abs(np.einsum('ij,ik->ijk', 
                            this_cl_zz_auto_diag, this_cl_zz_auto_diag)) + np.einsum('ijk,ijk->ijk'
                                                        , this_cl_object.cl_zz, this_cl_object.cl_zz))
    else:
        auto1_diag = np.einsum('ijj->ij', auto1_cl_zz.cl_zz)
        auto2_diag = np.einsum('ijj->ij', auto2_cl_zz.cl_zz)
        var = pre_factor[:,None,None]*np.abs(np.einsum('ij,ik->ijk', auto1_diag, auto2_diag))
        var += pre_factor[:,None,None]*this_cl_object.cl_zz**2
    std = var**0.5
    return std

pt.plot_cl_zpairs_vs_ell(CMASSxCMASS, compute_error_diagonals(CMASSxCMASS), 
                        [0,1,2,3,4], [0,1,2,3,4], ['z1=0.1', 'z1=0.3', 'z1=0.5', 'z1=0.7', 'z1=0.9'], 
                         ['z2=0.1', 'z2=0.3', 'z2=0.5', 'z2=0.7', 'z2=0.9'],
                        fname = 'temp.pdf')